# L1: Information retrieval

In this lab you will apply basic techniques from information retrieval to implement the core of a minimalistic search engine. The data for this lab consists of a collection of app descriptions scraped from the [Google Play Store](https://play.google.com/store/apps?hl=en). From this collection, your search engine should retrieve those apps whose descriptions best match a given query under the vector space model.

**⚠️ Make sure that you have read the [Rules for hand-in assignments](https://www.ida.liu.se/~TDDE16/exam.en.shtml#handins) and the [Policy on cheating and plagiarism](https://www.ida.liu.se/~TDDE16/exam.en.shtml#cheating) before starting with this lab.** 

## Data set

The app descriptions come in the form of a compressed [JSON](https://en.wikipedia.org/wiki/JSON) file. Start by loading this file into a [Pandas](https://pandas.pydata.org) [DataFrame](https://pandas.pydata.org/pandas-docs/stable/getting_started/dsintro.html#dataframe).

In [1]:
import bz2
import pandas as pd

with bz2.open('app-descriptions.json.bz2') as source:
    df = pd.read_json(source)

In Pandas, a DataFrame is a table with indexed rows and labelled columns of potentially different types. You can access data in a DataFrame in various ways, including by row and column. To give an example, the code in the next cell shows rows 200–204:

In [2]:
df[200:205]

,name,description
200,Brick Breaker Star: Space King,Introducing the best Brick Breaker game that e...
201,Brick Classic - Brick Game,Classic Brick Game!\n\nBrick Classic is a popu...
202,Bricks Breaker - Glow Balls,Bricks Breaker - Glow Balls is a addictive and...
203,Bricks Breaker Quest,How to play\n- The ball flies to wherever you ...
204,Brothers in Arms® 3,Fight brave soldiers from around the globe on ...


As you can see, there are two labelled columns: `name` (the name of the app) and `description` (a textual description). The code in the next cell shows how to access fields from the description column.

In [3]:
df['description'][200:205]

200    Introducing the best Brick Breaker game that e...
201    Classic Brick Game!\n\nBrick Classic is a popu...
202    Bricks Breaker - Glow Balls is a addictive and...
203    How to play\n- The ball flies to wherever you ...
204    Fight brave soldiers from around the globe on ...
Name: description, dtype: object

## Problem 1: Preprocessing

Your first task is to implement a preprocessor for your search engine. In the vector space model, *preprocessing* refers to any transformation applied to a text before vectorisation. Here you can restrict yourself to a simple type of preprocessing: tokenisation, stop word removal, and lemmatisation.

To implement your preprocessor, you can use [spaCy](https://spacy.io). Make sure to read the [Linguistic annotations](https://spacy.io/usage/spacy-101#annotations) section of the spaCy&nbsp;101; that section contains all the information you need for this problem (and more).

Implement your preprocessor by completing the skeleton code in the next cell, adding additional code as you deem necessary.

In [4]:
def preprocess(text):
    # TODO: Replace the next line with your own code.
    doc = nlp(text)
    return [token.lemma_ for token in doc if(not token.is_stop and token.lemma_.isalpha())]

Your implementation should conform to the following specification:

<strong>preprocess</strong> (<em>text</em>)

> Preprocesses the given text by tokenising it, removing any stop words, replacing each remaining token with its lemma (base form), and discarding all lemmas that contain non-alphabetical characters. Returns the list of remaining lemmas (represented as strings).

To speed up the preprocessing, you should disable loading those spaCy components you do not need, such as the parser, and named entity recogniser. See [here](https://spacy.io/usage/processing-pipelines#disabling) for more information about this.

### 🤞 Test your code

Test your implementation by running the following cell:

In [5]:
import spacy
nlp = spacy.load('en_core_web_sm')
preprocess('Apple is looking at buying U.K. startup for $1 billion')

['Apple', 'look', 'buy', 'startup', 'billion']

This should give the following output:


## Problem 2: Vectorising

Your next task is to vectorise the data – and more specifically, to map each app description to a tf–idf vector. For this you can use the [TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) class from [scikit-learn](https://scikit-learn.org/stable/). Make sure to specify your preprocessor from the previous problem as the `tokenizer` &ndash; not the `preprocessor`! &ndash; for the vectoriser. (In scikit-learn terminology, the `preprocessor` handles string-level preprocessing.)

In [6]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer as tfi
# TODO: Replace the next line with your own code.
vec = tfi(tokenizer=preprocess)
X = vec.fit_transform(df['description'])

### 🤞 Test your code

Test your implementation by running the following cell:

In [7]:
X.shape

(1614, 21396)

This should show the dimensions of the matrix `X` to be around 1614 × 21389. (The number of rows should be exactly 1614. The number of columns may differ from that given here depending on the version of spaCy and the version of the language model used, as well as the pre-processing.)

## Problem 3: Retrieving

To complete the search engine, your last task is to write a function that returns the most relevant app descriptions for a given query. An easy way to solve this task is to use scikit-learn&rsquo;s [NearestNeighbors](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestNeighbors.html) class. That class implements unsupervised nearest neighbours learning and allows you to easily find a predefined number of app descriptions whose vector representations are closest to the query vector.

In [8]:
from sklearn.neighbors import NearestNeighbors as near
algo = near(n_neighbors=20,metric='cosine')
algo.fit(X)
def search(query):
    # TODO: Replace the next line with your own code.
    query_res = vec.transform([query])
    idx = algo.kneighbors(query_res,return_distance=False).squeeze()
    return df.iloc[idx]

Your implementation should conform to the following specification:

<strong>search</strong> (<em>query</em>)

> Returns the 10 app descriptions most similar (in terms of cosine similarity) to the given query as a Pandas DataFrame.

### 🤞 Test your code

Test your implementation by running the following cell:

In [9]:
search('dodge trains')

,name,description
1301,Subway Surfers,DASH as fast as you can! \nDODGE the oncoming ...
1428,Train Conductor World,Master and manage the chaos of international r...
1394,Tiny Rails,All aboard for an adventure around the world!\...
1300,Subway Princess Runner,"Subway princess runner, Bus run, forest rush w..."
998,No Humanity - The Hardest Game,2M+ Downloads All Over The World!\n\n* IGN Nom...
1429,Train for Animals - BabyMagica free,"🚂 BabyMagica ""Train for Animals"" is a educatio..."
1168,Rush,Are you ready for a thrilling ride?\n\nRush th...
786,LEGO® DUPLO® Train,All aboard! Driving the colorful LEGO® DUPLO® ...
1465,Virus War - Space Shooting Game,Warning! Virus invasion! Destroy them with you...
1153,Road Riot,Road Riot is the global sensation that defined...


The top hit in the list should be *Subway Surfers*.

## Problem 4: Finding terms with low/high idf

Recall that the inverse document frequency (idf) of a term is the lower, the more documents from a given collection the term appears in. To get a better understanding for this concept, your next task is to write code to find out which terms from the app descriptions have the lowest/highest idf.

Start by sorting the terms in increasing order of idf, breaking ties by falling back on alphabetic order.

In [10]:
# TODO: Replace the next line with your own code.
idf = vec.idf_

#voca is a dictionary {words : showing up times}
voca = vec.vocabulary_
keys = voca.keys()

# idf first alphabetic next
terms = sorted(keys, key=lambda x: (idf[voca[x]],x))




Now, print the 10 terms with the lowest/highest idf. How do you explain the results?

In [11]:
print(terms[:10])
print(terms[-10:])
print(idf[voca['game']])
print(idf[voca['ﬁnd']])  #star
print(idf[voca['find']])


# strange thing here
# print(idf[voca['find']]) the word in line star is not 'find'
#Good eye! Yes, the word that is printed is not actually "find" but instead a version of it where the f and i are combined into a single character. 
#The same is true for the remainder of the non-Korean words at the end of the high-idf list.


['game', 'play', 'feature', 'free', 'new', 'world', 'time', 'app', 'fun', 'use']
['회원가입에', '회원을', '획득한', '효과', '효과음', 'ﬁnd', 'ﬁnger', 'ﬁnish', 'ﬁrst', 'ﬂye']
1.2970133998806652
7.6939430550968115
2.1947277461818846


The words in the first line can show up in different type of docs.
In the second line, korean will show up in only korean articles. But for other words in second line, they are not in 'standard format'.

## Problem 5: Keyword extraction

We often want to extract salient keywords from a document. A simple method is to pick the $k$ terms with the highest tf–idf value. Your last task in this lab is to implement this method. More specifically, we ask you to implement a function `keywords` that extracts keywords from a text.

In [16]:
import collections
def keywords(text, n=10):
    idf_vec = vec.transform([text]).toarray()
    idf_df =  pd.DataFrame({"tf_idf" : idf_vec.flatten(), "word" : vec.get_feature_names()})
    return idf_df.sort_values(by=['tf_idf', "word"],  ascending=False)["word"].tolist()[:n]

Your implementation should conform to the following specification:

<strong>keywords</strong> (<em>text</em>, <em>n</em> = 10)

> Returns a list with the $n$ (default value: 10) most salient keywords from the specified text, as measured by their tf–idf value relative to the collection of app descriptions.

### 🤞 Test your code

Test your implementation by running the following cell:

In [17]:
print(keywords(df['description'][1428]))

['train', 'railway', 'railroad', 'rail', 'chaos', 'crash', 'tram', 'timetable', 're', 'railyard']


E:\anaconda\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


This should give output similar to the following:

The exact output may differ slightly depending on the strategy used to break ties.

## Reflection questions

The following reflection questions will help you prepare for the diagnostic test. Answer each of them in the form of a short text and put your answers in the cell below. You will get feedback on your answers from your lab assistant.

**RQ 1.1:** Why do we remove common stop words and lemmatise the text? Can you give an example of a scenario where, in addition to common stopwords, there are also *application-specific* stop words?

**A 1.1**  
The stop words will show up many times in every single article. Here we want to have some statistics features of articles, so it's better to remove those common features.   
Especially in this lab, those stop words can not help us narrow tthe search results. Removing them can accelerate the processing.  

The scenario is : in deep learning conference, it;s better to stop words like 'deep learning' 'deeplearning'

**v2**  
Stop words are non-informative and they are memory overhead. Stop words are useless while doing sentiment analysis. For example: "Mr", "Teacher","Doctor". lemmatization of text is grouping different forms of words into root form, having the same meaning. For example: "looking", "looked" to "look".

**RQ 1.2:** In Problem&nbsp;2, what do the dimensions of the matrix `X` correspond to? This matrix gives rise to different types of *representations*. Explain what these representations are. What information from the data is preserved, what information is lost in these representations?

**A 1.2**  
The shape of X is (1614,21396).  
1614 is the number of discriptions (the number of data).  
21396 is the number of unique words.  
Every document here is transformed to a vector of tfidf values. In this form, we abstract words frequency information but we lose the linguitic relationship between words.  

**RQ 1.3:** What does it mean that a term has a high/low idf value? Based on this, how can we use idf to automatically identify stop words? Why do you think is idf not used as a term weighting on its own, but always in connection with term frequency (tf–idf)?

**A 1.3**  
high idf means words are only frequent in some specific documents.Low idf means opposite.  

Stop words will show up frequently in almost all documents. So, the idf value of stop words will be veryu low.  

**v1**
Idf only tells how many documents contain the term but has nothing to do with frequency. TF-IDF can tell us important words to identify documents.

**v2**
IDF does not consider semantic correlations between the terms, which may lead to less relevant information retrieval. 

**Congratulations on finishing L1! 👍**